<a href="https://colab.research.google.com/github/PaiwandAlAhmady/RoboCup-Work_Object-Segmentation-Model/blob/main/YOLO11_seg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOv11 Notebook for RoboCup@Work

## Relevant Dependencies

In [ ]:
%pip install -q ultralytics -U
%pip install -q depthai==3.0.0rc2 -U
%pip install -q depthai-nodes==0.3.0 -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.3/224.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 17.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Lädt Trainingsdatensatz von Drive auf lokalen Colab Speicher -> effizienter
!cp -r /content/drive/MyDrive/RoboCup_Trainingsdatensatz /content/

cp: cannot stat '/content/drive/MyDrive/RoboCup_Trainingsdatensatz': No such file or directory


In [ ]:
from ultralytics import YOLO

model_yolo_11_n = YOLO("yolo11n-seg.pt")
model_yolo_11_s = YOLO("yolo11s-seg.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Check Data

In [ ]:
import os

BASE = "/content/drive/MyDrive/RoboCup_Trainingsdatensatz"
VALID_IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}

def check_split(split):
    img_dir = os.path.join(BASE, split, "images")
    lbl_dir = os.path.join(BASE, split, "labels")

    if not os.path.exists(img_dir) or not os.path.exists(lbl_dir):
        return None

    img_files = [f for f in os.listdir(img_dir) if os.path.splitext(f)[1].lower() in VALID_IMG_EXTS]
    lbl_files = [f for f in os.listdir(lbl_dir) if f.endswith(".txt")]

    img_bases = {os.path.splitext(f)[0] for f in img_files}
    lbl_bases = {os.path.splitext(f)[0] for f in lbl_files}

    missing_labels = img_bases - lbl_bases
    missing_images = lbl_bases - img_bases

    return {
        "split": split,
        "images": len(img_files),
        "labels": len(lbl_files),
        "aligned": len(missing_labels) == 0 and len(missing_images) == 0,
        "missing_labels": len(missing_labels),
        "missing_images": len(missing_images),
        "example_missing_labels": list(missing_labels)[:5],
        "example_missing_images": list(missing_images)[:5],
    }

splits = ["train", "valid", "test"]
results = [check_split(s) for s in splits]
results = [r for r in results if r is not None]

print("=== Dataset Integrity Report ===")
for r in results:
    print(f"\nSplit: {r['split']}")
    print(f"  Images: {r['images']} | Labels: {r['labels']}")
    print(f"  All aligned? {r['aligned']}")
    print(f"  Missing labels: {r['missing_labels']}  {r['example_missing_labels']}")
    print(f"  Missing images: {r['missing_images']}  {r['example_missing_images']}")


=== Dataset Integrity Report ===

Split: train
  Images: 1912 | Labels: 1912
  All aligned? True
  Missing labels: 0  []
  Missing images: 0  []

Split: valid
  Images: 490 | Labels: 490
  All aligned? True
  Missing labels: 0  []
  Missing images: 0  []

Split: test
  Images: 0 | Labels: 0
  All aligned? True
  Missing labels: 0  []
  Missing images: 0  []


In [ ]:
import os
from collections import Counter

BASE = "/content/drive/MyDrive/RoboCup_Trainingsdatensatz"
SPLITS = ["train", "valid", "test"]

CLASS_NAMES = {
    0: "Kugellager",
    1: "Motor",
    2: "Schraubenzieher"
}

def count_classes_in_split(split):
    lbl_dir = os.path.join(BASE, split, "labels")
    if not os.path.exists(lbl_dir):
        return None

    counts = Counter()
    total_labels = 0

    for f in os.listdir(lbl_dir):
        if not f.endswith(".txt"):
            continue
        with open(os.path.join(lbl_dir, f), "r") as fh:
            for line in fh:
                parts = line.strip().split()
                if not parts:
                    continue
                try:
                    cls = int(float(parts[0]))
                    counts[cls] += 1
                    total_labels += 1
                except ValueError:
                    continue

    return split, counts, total_labels

print("=== Label Class Distribution ===")
grand_total = Counter()
for split in SPLITS:
    result = count_classes_in_split(split)
    if result is None:
        continue
    split, counts, total_labels = result
    print(f"\nSplit: {split}")
    print(f" Total labels: {total_labels}")
    for cls_id, name in CLASS_NAMES.items():
        print(f"   {cls_id} ({name}): {counts[cls_id]}")
    grand_total.update(counts)

print("\n=== Overall dataset ===")
for cls_id, name in CLASS_NAMES.items():
    print(f" {cls_id} ({name}): {grand_total[cls_id]}")
print(f" TOTAL: {sum(grand_total.values())}")


=== Label Class Distribution ===

Split: train
 Total labels: 2371
   0 (Kugellager): 779
   1 (Motor): 760
   2 (Schraubenzieher): 832

Split: valid
 Total labels: 566
   0 (Kugellager): 187
   1 (Motor): 169
   2 (Schraubenzieher): 210

Split: test
 Total labels: 0
   0 (Kugellager): 0
   1 (Motor): 0
   2 (Schraubenzieher): 0

=== Overall dataset ===
 0 (Kugellager): 966
 1 (Motor): 929
 2 (Schraubenzieher): 1042
 TOTAL: 2937


## Failed Run on Offline Augmentation Data

In [ ]:
results = model.train(data="/content/Trainingsdatensatz_RoboCup_aug/data.yaml", epochs=150, imgsz=640)

Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Trainingsdatensatz_RoboCup_aug/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspect

In [ ]:
#  results = model.train(
#      data="/content/Trainingsdatensatz_RoboCup_aug/data.yaml",
#      epochs=150,
#      imgsz=640,
#      project="/content/drive/MyDrive/YOLO_Runs",  # fester Speicherort auf Drive
#      name="train_thesis_run1",                     # eindeutiger Run-Name
#      exist_ok=True,
#      patience=50,   # EarlyStopping patience
#      save=True,
#      save_period=10 # optional: zusätzlich alle 10 Epochen sichern
#  )

## Model

In [ ]:
from ultralytics import YOLO
from datetime import datetime
from google.colab import runtime

data_yaml = "/content/RoboCup_Trainingsdatensatz/data.yaml"
common = dict(
    data=data_yaml,
    imgsz=832, batch=32, workers=4,
    epochs=300, patience=60,
    cache=True, deterministic=True, seed=0,
    save=True, save_period=25,
    # Augmentations:
    fliplr=0.5, flipud=0.10, degrees=10.0, shear=3.0, perspective=0.000,
    scale=0.50, translate=0.20,
    hsv_h=0.02, hsv_s=0.80, hsv_v=0.50,
    mosaic=0.8, close_mosaic=10, mixup=0.10, copy_paste=0.20,
    erasing=0.50
)

# 1) n-seg
run_n = f"y11nseg_832_b32_pat60_{datetime.now().strftime('%Y%m%d_%H%M')}"
model_n = YOLO("yolo11n-seg.pt")  # oder .yaml, falls from-scratch
model_n.train(project="/content/drive/MyDrive/YOLO_Runs/segment", name=run_n, **common)

runtime.unassign()

Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.2, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/RoboCup_Trainingsdatensatz/data.yaml, degrees=10.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.5, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.02, hsv_s=0.8, hsv_v=0.5, imgsz=832, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11n-seg.pt, momentum=0.937, mosaic=0.8, multi_scale=False, name=y11nseg_832_b32_pat60_20251001_16492, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, ove

KeyboardInterrupt: 

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
from ultralytics import YOLO
from datetime import datetime
from google.colab import runtime

data_yaml = "/content/RoboCup_Trainingsdatensatz/data.yaml"
common = dict(
    data=data_yaml,
    imgsz=832, batch=32, workers=4,
    epochs=300, patience=60,
    cache=True, deterministic=True, seed=0,
    save=True, save_period=25,
    # Augmentations:
    fliplr=0.5, flipud=0.10, degrees=10.0, shear=3.0, perspective=0.000,
    scale=0.50, translate=0.20,
    hsv_h=0.02, hsv_s=0.80, hsv_v=0.50,
    mosaic=0.8, close_mosaic=10, mixup=0.10, copy_paste=0.20,
    erasing=0.50
)

# 2) s-seg
run_s = f"y11sseg_832_b32_pat60_{datetime.now().strftime('%Y%m%d_%H%M')}"
model_s = YOLO("yolo11s-seg.pt")  # oder .yaml
model_s.train(project="/content/drive/MyDrive/YOLO_Runs/segment", name=run_s, **common)

runtime.unassign()

Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.2, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/RoboCup_Trainingsdatensatz/data.yaml, degrees=10.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.5, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.02, hsv_s=0.8, hsv_v=0.5, imgsz=832, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11s-seg.pt, momentum=0.937, mosaic=0.8, multi_scale=False, name=y11sseg_832_b32_pat60_20251001_2215, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, over

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fcdc9273fe0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.04

## Convertion for further Purposes

In [ ]:
%pip uninstall -y numpy
%pip install -U "numpy<2.0" "luxonis-ml[data]" modelconv==0.4.0
%pip install -U depthai==3.0.0rc2 depthai-nodes==0.3.0
# Runtime hart neu starten, damit die neuen Wheels geladen werden:
import os, sys; os.kill(os.getpid(), 9)

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 81.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of luxonis-ml[data] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of s3transfer to determine which version is

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.3/224.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.5 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86


In [ ]:
import shutil

shutil.copy("/content/drive/MyDrive/YOLO_Runs (2)/segment/y11nseg_832_b32_pat60_20251001_16492/weights/best.pt", "yolo11n_instance_segmentation_trained.pt")

'yolo11n_instance_segmentation_trained.pt'

In [ ]:
HUBAI_API_KEY = "tapi.S4zbE9uV-5yAedJy2F2ZAQ.qPQAXWC-Rvbw_qTs0N7nvbd7Va0i6lRc2hxMpx6Web4GMK2OO2JAI8fbAQNGatDal9rGB9GFWJkPCiYjyONl1Q"

In [ ]:
from modelconverter import convert

converted_model = convert.RVC2(
    api_key=HUBAI_API_KEY,
    path="/content/yolo11n_instance_segmentation_trained.pt",
    name="YOLO11 Nano RoboCup Instance Segmentation",
    description_short="x",
    yolo_version="yolov11",
    yolo_input_shape="832 832",
    yolo_class_names=["Kugellager", "Motor", "Schraubenzieher"],
    tasks=["INSTANCE_SEGMENTATION"],
    license_type="MIT",
    is_public=False
)

INFO     Detected PyTorch model. Only YOLO models are supported.                                      ]8;id=841434;file:///usr/local/lib/python3.12/dist-packages/modelconverter/utils/config.py\config.py]8;;\:]8;id=473820;file:///usr/local/lib/python3.12/dist-packages/modelconverter/utils/config.py#308\308]8;;\

Model variant 'YOLO11 Nano RoboCup Instance Segmentation' created with ID '760bb7ca-985c-4eda-ad82-47b31cd13755'

Model instance 'YOLO11 Nano RoboCup Instance Segmentation base instance' created with ID 
'4dcc18e1-ad8c-4727-9708-3ac651cf1812'

File '/content/yolo11n_instance_segmentation_trained.pt' uploaded to model instance 
'4dcc18e1-ad8c-4727-9708-3ac651cf1812'

Model instance 'YOLO11 Nano RoboCup Instance Segmentation exported to Target.RVC2' created for RVC2 export with ID 
'3cde516c-b6f6-49fe-8d12-f605999cd535'

Output()

Output()

Downloaded 'yolo11n_instance_segmentation_trained.rvc2.tar.xz'

In [ ]:
from modelconverter import convert

converted_model = convert.RVC2(
    api_key=HUBAI_API_KEY,
    path="/content/yolo11n_instance_segmentation_trained.pt",
    name="YOLO11n Seg 640",
    description_short="RVC2 640x640",
    yolo_version="yolov11",
    yolo_input_shape="640 640",
    yolo_class_names=["Kugellager", "Motor", "Schraubenzieher"],
    tasks=["INSTANCE_SEGMENTATION"],
    license_type="MIT",
    is_public=False,
    # optional, falls dein Converter cached/überschreibt:
    # force_rebuild=True, overwrite=True
)

INFO     Detected PyTorch model. Only YOLO models are supported.                                      ]8;id=376091;file:///usr/local/lib/python3.12/dist-packages/modelconverter/utils/config.py\config.py]8;;\:]8;id=140122;file:///usr/local/lib/python3.12/dist-packages/modelconverter/utils/config.py#308\308]8;;\

Model 'YOLO11n Seg 640' created with ID '313ea2e9-eb9a-4594-b9f2-552cc7786e98'

Model variant 'YOLO11n Seg 640' created with ID 'bc59b9fe-dd69-40a5-936d-70c0284e9041'

Model instance 'YOLO11n Seg 640 base instance' created with ID '6ee2c512-38a2-4953-85e1-cc4e6b75c140'

File '/content/yolo11n_instance_segmentation_trained.pt' uploaded to model instance 
'6ee2c512-38a2-4953-85e1-cc4e6b75c140'

Model instance 'YOLO11n Seg 640 exported to Target.RVC2' created for RVC2 export with ID 
'af951bf5-5b9b-4bb7-b64d-18a97153fe2b'

Output()

Output()

Downloaded 'yolo11n_instance_segmentation_trained.rvc2.tar.xz'

In [ ]:
from ultralytics import YOLO
m = YOLO("/content/drive/MyDrive/YOLO_Runs/segment/y11nseg_832_b32_pat60_20251001_16492/weights/best.pt")
m.export(format="onnx", imgsz=832)  # nimm deine Trainings-Size (z.B. 832 oder 640)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.207 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLO11n-seg summary (fused): 113 layers, 2,835,153 parameters, 0 gradients, 9.6 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/YOLO_Runs/segment/y11nseg_832_b32_pat60_20251001_16492/weights/best.pt' with input shape (1, 3, 832, 832) BCHW and output shape(s) ((1, 39, 14196), (1, 32, 208, 208)) (5.8 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim>=0.1.67', 'onnxruntime'] not found, attempting AutoUpdate...

requirements: AutoUpda

'/content/drive/MyDrive/YOLO_Runs/segment/y11nseg_832_b32_pat60_20251001_16492/weights/best.onnx'

In [ ]:
from ultralytics import YOLO

# Beispiel: nimm YOLOv11n-Seg (kleinste Version)
model = YOLO("/content/drive/MyDrive/YOLO_Runs/segment/y11nseg_832_b32_pat60_20251001_16492/weights/best.pt")

# Optional: kurzer Test
model.info()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
YOLO11n-seg summary: 203 layers, 2,843,193 parameters, 0 gradients, 9.7 GFLOPs


(203, 2843193, 0, 9.728768)